https://cloud.google.com/vision/docs

In [1]:
def async_detect_document(gcs_source_uri, gcs_destination_uri):
    """OCR with PDF/TIFF as source files on GCS"""
    import re
    from google.cloud import vision
    from google.cloud import storage
    from google.protobuf import json_format
    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = 'application/pdf'

    # How many pages should be grouped into each json output file.
    batch_size = 2

    client = vision.ImageAnnotatorClient()

    feature = vision.types.Feature(
        type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
    input_config = vision.types.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.types.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size)

    async_request = vision.types.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = client.async_batch_annotate_files(
        requests=[async_request])

    print('Waiting for the operation to finish.')
    operation.result(timeout=420)

    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    storage_client = storage.Client()

    match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
    bucket_name = match.group(1)
    prefix = match.group(2)

    bucket = storage_client.get_bucket(bucket_name)

    # List objects with the given prefix.
    blob_list = list(bucket.list_blobs(prefix=prefix))
    print('Output files:')
    for blob in blob_list:
        print(blob.name)

    # Process the first output file from GCS.
    # Since we specified batch_size=2, the first response contains
    # the first two pages of the input file.
    output = blob_list[0]

    json_string = output.download_as_string()
    response = json_format.Parse(
        json_string, vision.types.AnnotateFileResponse())

    # The actual response for the first page of the input file.
    first_page_response = response.responses[0]
    annotation = first_page_response.full_text_annotation

    # Here we print the full text from the first page.
    # The response contains more information:
    # annotation/pages/blocks/paragraphs/words/symbols
    # including confidence scores and bounding boxes
    print(u'Full text:\n{}'.format(
        annotation.text))

In [ ]:
async_detect_document('gs://mydatafiles/fw8ben.pdf', 'gs://mydatafiles/test.json')

In [2]:
async_detect_document('gs://mydatafiles/01_input/Primarschule.pdf','gs://mydatafiles/01_input/Primarschule.json')

Waiting for the operation to finish.
Output files:
01_input/Primarschule.jsonoutput-1-to-2.json
01_input/Primarschule.jsonoutput-3-to-4.json
Full text:
stadt
<' widenswil
Primarschule
lernen begreifen erfahren
P.P. A CH-8820 Wädenswil Post CH AG
Familie
Vivek Maswadkar und
Aruna Krishnamurthy
General-Werdmüller-Strasse 14
8804 Au ZH
Schulverwaltung
Schönenbergstrasse 4a
Postfach
8820 Wädenswil
Direkt 044 789 74 45
Telefon 044 789 74 40
daniel.spiess@waedenswil.ch
www.pswaedenswil.ch
Wädenswil, 6. August 2020
08.15 dsp
Bestätigung für die ausserfamiliäre Betreuung im Schuljahr 2020/21
Sehr geehrte Frau Krishnamurthy, sehr geehrter Herr Maswadkar
Gerne bestätigen wir Ihnen den Aufenthalt Ihres Kindes / Ihrer Kinder im Schülerclub /
Mittagstisch im Schuljahr 2020/21 wie folgt:
Name
Modul
Eintritt
Wochentage Tarif / Woche Standort
Aaryan
Aaryan
C Nachmittagsclub neu
Mahlzeiten
18.08.2020
18.08.2020
Mo Di Mi Do
Mo Di Do
212.00
48.00
Steinacher 1
Steinacher 1
Bei der Erstanmeldung wird eine 

In [4]:
text="stadt\n\u003c' widenswil\nPrimarschule\nlernen begreifen erfahren\nP.P. A CH-8820 Wädenswil Post CH AG\nFamilie\nVivek Maswadkar und\nAruna Krishnamurthy\nGeneral-Werdmüller-Strasse 14\n8804 Au ZH\nSchulverwaltung\nSchönenbergstrasse 4a\nPostfach\n8820 Wädenswil\nDirekt 044 789 74 45\nTelefon 044 789 74 40\ndaniel.spiess@waedenswil.ch\nwww.pswaedenswil.ch\nWädenswil, 6. August 2020\n08.15 dsp\nBestätigung für die ausserfamiliäre Betreuung im Schuljahr 2020/21\nSehr geehrte Frau Krishnamurthy, sehr geehrter Herr Maswadkar\nGerne bestätigen wir Ihnen den Aufenthalt Ihres Kindes / Ihrer Kinder im Schülerclub /\nMittagstisch im Schuljahr 2020/21 wie folgt:\nName\nModul\nEintritt\nWochentage Tarif / Woche Standort\nAaryan\nAaryan\nC Nachmittagsclub neu\nMahlzeiten\n18.08.2020\n18.08.2020\nMo Di Mi Do\nMo Di Do\n212.00\n48.00\nSteinacher 1\nSteinacher 1\nBei der Erstanmeldung wird eine Einschreibegebühr und bei einer Änderung der Module und / oder Tage\nwährend des Schuljahres eine Bearbeitungsgebühr von jeweils CHF 30.00 in Rechnung gestellt.\nFreundlich grüsst\nStadt Wädenswil\nDaniel Spiess\nSachbearbeiter\nbilden und forschen\nwädenswil\n"

In [5]:
print(text)

stadt
<' widenswil
Primarschule
lernen begreifen erfahren
P.P. A CH-8820 Wädenswil Post CH AG
Familie
Vivek Maswadkar und
Aruna Krishnamurthy
General-Werdmüller-Strasse 14
8804 Au ZH
Schulverwaltung
Schönenbergstrasse 4a
Postfach
8820 Wädenswil
Direkt 044 789 74 45
Telefon 044 789 74 40
daniel.spiess@waedenswil.ch
www.pswaedenswil.ch
Wädenswil, 6. August 2020
08.15 dsp
Bestätigung für die ausserfamiliäre Betreuung im Schuljahr 2020/21
Sehr geehrte Frau Krishnamurthy, sehr geehrter Herr Maswadkar
Gerne bestätigen wir Ihnen den Aufenthalt Ihres Kindes / Ihrer Kinder im Schülerclub /
Mittagstisch im Schuljahr 2020/21 wie folgt:
Name
Modul
Eintritt
Wochentage Tarif / Woche Standort
Aaryan
Aaryan
C Nachmittagsclub neu
Mahlzeiten
18.08.2020
18.08.2020
Mo Di Mi Do
Mo Di Do
212.00
48.00
Steinacher 1
Steinacher 1
Bei der Erstanmeldung wird eine Einschreibegebühr und bei einer Änderung der Module und / oder Tage
während des Schuljahres eine Bearbeitungsgebühr von jeweils CHF 30.00 in Rechnung ge

In [10]:
text="Packliste für den Schülerclub\n• Stundenplan\n• Zahnbürste\n• Rutschfeste Hausschuhe\nWettergerechte Kleidung\n• Ersatzkleidung\n• Evtl. „Übergangsobjekt\" für jüngere Kinder (.Kuscheltier“ o. ähnliches.)\nIm Frühling/Sommer ausserdem\n• Sonnencreme\n• Sonnenhut\n• Nach Bedarf Badesachen\nTipp: Orientierungshilfe für das Kind zu Beginn des neuen Schuljahres\nAnhänger oder ähnlicher Gegenstand an den Tagen mitgeben, an denen das Kind den Schülerclub besucht\n(Schlüsselanhänger, Symbol etc.)\n"

In [11]:
print(text)

Packliste für den Schülerclub
• Stundenplan
• Zahnbürste
• Rutschfeste Hausschuhe
Wettergerechte Kleidung
• Ersatzkleidung
• Evtl. „Übergangsobjekt" für jüngere Kinder (.Kuscheltier“ o. ähnliches.)
Im Frühling/Sommer ausserdem
• Sonnencreme
• Sonnenhut
• Nach Bedarf Badesachen
Tipp: Orientierungshilfe für das Kind zu Beginn des neuen Schuljahres
Anhänger oder ähnlicher Gegenstand an den Tagen mitgeben, an denen das Kind den Schülerclub besucht
(Schlüsselanhänger, Symbol etc.)

